In [2]:
# Team 1 LendingClub primary notebook
# import dependencies
import pandas as pd
import numpy as np # needed for replace nan code below
import glob

# pull in all csvs and merge them into a single dataframe
bigdf = pd.concat([pd.read_csv(f, low_memory=False) for f in
                    glob.glob('data/LoanStats*.csv')], ignore_index=True)

In [16]:
# drop irrelevant rows
data = bigdf.drop([
                "id", "member_id", "url", "desc", "hardship_type", "hardship_reason",
                "hardship_status", "deferral_term", "hardship_amount",
                "hardship_start_date", "hardship_end_date", "payment_plan_start_date",
                "hardship_length", "hardship_dpd", "hardship_loan_status",
                "orig_projected_additional_accrued_interest",
                "hardship_payoff_balance_amount", "hardship_last_payment_amount",
                "debt_settlement_flag_date", "settlement_status", "settlement_date",
                "settlement_amount", "settlement_percentage", "settlement_term","grade",\
                "sub_grade","emp_title","debt_settlement_flag","disbursement_method","verification_status",\
                "url","desc","zip_code","earliest_cr_line","inq_last_6mths","mths_since_last_record",\
                "pub_rec","revol_bal","revol_util","total_acc","initial_list_status","out_prncp","out_prncp_inv",\
                "total_pymnt","total_pymnt_inv","total_rec_prncp","total_rec_int","recoveries","collection_recovery_fee",\
                "last_pymnt_d","last_pymnt_amnt","next_pymnt_d","last_credit_pull_d","collections_12_mths_ex_med",\
                "policy_code","application_type","annual_inc_joint","dti_joint","verification_status_joint","acc_now_delinq",\
                "tot_coll_amt","tot_cur_bal","open_acc_6m","open_act_il","open_il_12m","open_il_24m","mths_since_rcnt_il",\
                "total_bal_il","il_util","open_rv_12m","open_rv_24m","max_bal_bc","all_util","total_rev_hi_lim","inq_fi",\
                "total_cu_tl","inq_last_12m","acc_open_past_24mths","avg_cur_bal","bc_open_to_buy","bc_util","chargeoff_within_12_mths",\
                "delinq_amnt","mo_sin_old_il_acct","mo_sin_old_rev_tl_op","mo_sin_rcnt_rev_tl_op","mo_sin_rcnt_tl","mort_acc",\
                "mths_since_recent_bc","mths_since_recent_bc_dlq","mths_since_recent_inq","mths_since_recent_revol_delinq","num_accts_ever_120_pd",\
                "num_actv_bc_tl","num_actv_rev_tl","num_bc_sats","num_bc_tl","num_il_tl","num_op_rev_tl","num_rev_accts","num_rev_tl_bal_gt_0",\
                "num_sats","num_tl_120dpd_2m","num_tl_30dpd","num_tl_90g_dpd_24m","num_tl_op_past_12m","pct_tl_nvr_dlq","percent_bc_gt_75",\
                "pub_rec_bankruptcies","tax_liens","tot_hi_cred_lim","total_bal_ex_mort","total_bc_limit","total_il_high_credit_limit",\
                "revol_bal_joint","sec_app_earliest_cr_line","sec_app_inq_last_6mths","sec_app_mort_acc","sec_app_open_acc","sec_app_revol_util",\
                "sec_app_open_act_il","sec_app_num_rev_accts","sec_app_chargeoff_within_12_mths","sec_app_collections_12_mths_ex_med",\
                "sec_app_mths_since_last_major_derog","hardship_flag","hardship_type","hardship_reason","hardship_status","deferral_term",\
                "hardship_amount","hardship_start_date","hardship_end_date","payment_plan_start_date","hardship_length","hardship_dpd",\
                "hardship_loan_status","orig_projected_additional_accrued_interest","hardship_payoff_balance_amount","hardship_last_payment_amount",\
                "disbursement_method","debt_settlement_flag","debt_settlement_flag_date","settlement_status","settlement_date","settlement_amount",\
                "settlement_percentage","settlement_term","mths_since_last_major_derog"], axis=1)

In [17]:
data.head()

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,home_ownership,annual_inc,issue_d,loan_status,pymnt_plan,purpose,title,addr_state,dti,delinq_2yrs,mths_since_last_delinq,open_acc,total_rec_late_fee
0,30000.0,30000.0,30000.0,36 months,7.34%,930.99,3 years,OWN,95000.0,Mar-18,Fully Paid,n,house,Home buying,TX,16.18,1.0,10.0,10.0,0.0
1,34825.0,34825.0,34825.0,60 months,12.61%,785.45,2 years,MORTGAGE,125000.0,Mar-18,Current,n,credit_card,Credit card refinancing,AZ,21.31,0.0,45.0,12.0,0.0
2,2600.0,2600.0,2600.0,36 months,7.96%,81.43,3 years,MORTGAGE,62000.0,Mar-18,Current,n,other,Other,MI,19.61,0.0,NaN,8.0,0.0
3,20000.0,20000.0,20000.0,60 months,9.92%,424.16,3 years,MORTGAGE,110000.0,Mar-18,Current,n,home_improvement,Home improvement,VA,10.56,0.0,44.0,9.0,0.0
4,17000.0,17000.0,17000.0,60 months,20.39%,454.10,10+ years,RENT,52000.0,Mar-18,Current,n,debt_consolidation,Debt consolidation,FL,15.65,0.0,77.0,9.0,0.0


In [ ]:
mydf= pd.read_csv()

In [ ]:
data_merge